## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [5]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-05-15 16:10:03--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  29.3MB/s    in 8.6s    

2020-05-15 16:10:12 (29.0 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,AveragePooling2D,Flatten,Dropout

Using TensorFlow backend.


In [0]:
batch_size=100
num_classes=2

In [0]:
datagenerator=ImageDataGenerator(preprocessing_function=preprocess_input)


In [10]:
train_gen=datagenerator.flow_from_directory('concrete_data_week4/train',target_size=(224,224),batch_size=batch_size)
valid_gen=datagenerator.flow_from_directory('concrete_data_week4/valid',target_size=(224,224),batch_size=batch_size)


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [11]:
model=Sequential()
model.add(VGG16(weights='imagenet',include_top=False,pooling='avg'))
model.layers[0].trainable = False



model.add(Dense(256,activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.summary()

58892288/58889256 [==============================] - 3s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 14,846,530
Trainable params: 131,842
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(train_gen)
steps_per_epoch_validation = len(valid_gen)
num_epochs = 2

In [14]:
H=model.fit_generator(train_gen,epochs=2,validation_data=valid_gen)

Epoch 1/2
301/301 [==============================] - 371s 1s/step - loss: 0.0351 - accuracy: 0.9866 - val_loss: 1.8954e-05 - val_accuracy: 0.9977
Epoch 2/2
301/301 [==============================] - 357s 1s/step - loss: 0.0074 - accuracy: 0.9976 - val_loss: 7.4741e-05 - val_accuracy: 0.9981


In [0]:
model.save('classifier_vgg16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [0]:
from keras.models import load_model
resnet_model=load_model('classifier_resnet_model.h5')
vgg16_model=load_model('classifier_vgg16_model.h5')

In [0]:
datagent=ImageDataGenerator()

In [19]:
test_gen=datagent.flow_from_directory("concrete_data_week4/test",target_size=(224,224),shuffle=False)

Found 500 images belonging to 2 classes.


In [21]:
print("The Performance by vgg16 model is below")
vgg=vgg16_model.evaluate_generator(test_gen)
print(vgg)
print("Loss is ",str(vgg[0]))
print("The accuracy is ",str(vgg[1]))

The Performance by vgg16 model is below
[0.06328427046537399, 0.9800000190734863]
Loss is  0.06328427046537399
The accuracy is  0.9800000190734863


In [23]:
print("The Performance by resnet50 model is below")
resnet=resnet_model.evaluate_generator(test_gen)
print(resnet)
print("Loss is ",str(resnet[0]))
print("The accuracy is ",str(resnet[1]))

The Performance by resnet50 model is below
[3.488461971282959, 0.5419999957084656]
Loss is  3.488461971282959
The accuracy is  0.5419999957084656


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set.

Use the following cells to make your predictions.

In [0]:
vgg_pred=vgg16_model.predict_generator(test_gen)
res_pred=resnet_model.predict_generator(test_gen)


In [0]:
def pred(x):
    for i in x:
        j=np.argmax(i)
        if(j==0):
            print("Negetive")
        else:
            print("Positive")

Resnet50 predictions for first 5 in testset

In [31]:
pred(res_pred[0:5])

Negetive
Negetive
Negetive
Negetive
Negetive


Vgg16 predictions for first 5 in testset

In [32]:
pred(vgg_pred[0:5])

Negetive
Negetive
Negetive
Negetive
Negetive


original predictions for first 5 in testset  

In [33]:
pred(test_gen.next()[1][0:5])

Negetive
Negetive
Negetive
Negetive
Negetive


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).